In [1]:
import sys
sys.path.append('..') # for import src

import os
import cloudpickle
import lzma
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import ccxt

import src
cloudpickle.register_pickle_by_value(src) # for model portability

In [2]:
import numpy as np
import pandas as pd
import time
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

def execute_query(client, q):
    query = gql('query Query {}'.format(q))
    return client.execute(query)

In [3]:
weth_id = '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'

def fetch_pair(client, created_at=0, weth_is_token1=False):
    tmpl = """
    {{
      pairs(orderBy: createdAtTimestamp, where: {{ createdAtTimestamp_gte: {}, token{}: "{}" }}, first: 1000) {{
        id
        createdAtTimestamp
        token0 {{
          id
          symbol
        }}
        token1 {{
          id
          symbol
        }}
      }}
    }}
    """
    return execute_query(client, tmpl.format(created_at, 1 if weth_is_token1 else 0, weth_id))

def do_fetch_all_pairs(client, created_at=1, weth_is_token1=False):
    dfs = []
    
    while True:
        print('created_at', created_at)
        try:
            res = fetch_pair(client, created_at, weth_is_token1)
        except Exception as e:
            print(e)
            print('retry')
            time.sleep(10)
            continue
        if len(res['pairs']) == 0:
            break
        print('len', len(res['pairs']))
        for row in res['pairs']:
            row['token0_id'] = row['token0']['id']
            row['token0_symbol'] = row['token0']['symbol']
            row['token1_id'] = row['token1']['id']
            row['token1_symbol'] = row['token1']['symbol']
        df = pd.DataFrame(res['pairs']).drop(columns=['token0', 'token1'])
        dfs.append(df)
        created_at = df['createdAtTimestamp'].max()
        if len(res['pairs']) < 1000:
            break

    df = pd.concat(dfs).drop_duplicates()
    df = df.rename(columns={
        'createdAtTimestamp': 'timestamp'
    })
    df['timestamp'] = df['timestamp'].astype('int')
    df = df.set_index(['id']).sort_index()
    return df

def fetch_all_pairs(client, created_at=1):
    dfs = [
        do_fetch_all_pairs(client, created_at, False),
        do_fetch_all_pairs(client, created_at, True),
    ]
    return pd.concat(dfs).sort_index()

transport = RequestsHTTPTransport(url="https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2")
client = Client(transport=transport, fetch_schema_from_transport=True)

df_pair = fetch_all_pairs(client, 1668000000)
print('index duplicated count {}'.format(df_pair[df_pair.index.duplicated(keep='last')].shape[0]))
display(df_pair)
df_pair.to_pickle('/tmp/df_pair.pkl')

created_at 1668000000
{'message': 'Failed to get entities from store: canceling statement due to conflict with recovery, query = /* controller=\'filter\',application=\'sgd213744\',route=\'c9b0081408fe305-550b53256d13c577\',action=\'15973077\' */\nselect \'Token\' as entity, to_jsonb(c.*) as data from (\nselect c.*, p.id::text as g$parent_id\n/* child_type_d */ from rows from (unnest($1), unnest($2::text[])) as p(id, child_id), "sgd213744"."token" c where c.block_range @> $3 and coalesce(upper("block_range"), 2147483647) > $4 and lower("block_range") <= $5 and c.id = p.child_id limit 283) c\n  -- binds: [["..."], 15973077, 15973077, 15973077]'}
retry


KeyboardInterrupt: 

In [4]:

class PairFetcher:
    def __init__(self):
        self.keys = {}
        self.data_id = 'univ2_pair'
        
    def fetch(self, last_timestamp=None):
        transport = RequestsHTTPTransport(url="https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2")
        client = Client(transport=transport, fetch_schema_from_transport=True)
        
        df = fetch_all_pairs(client, 1 if last_timestamp is None else last_timestamp + 1)
        
        df = df.loc[df['timestamp'] < df['timestamp'].max() - 300] # buffer
        
        return df


In [22]:
fetcher = PairFetcher()
df = fetcher.fetch(last_timestamp=None)
display(df)
df = fetcher.fetch(last_timestamp=1668000000)
display(df)

created_at 1
len 1000
created_at 1597546980
len 1000
created_at 1600545834
len 1000
created_at 1602639738
len 1000
created_at 1604158136
{'message': 'Failed to get entities from store: canceling statement due to conflict with recovery, query = /* controller=\'filter\',application=\'sgd213744\',route=\'c9b0081408fe305-baa7bbcd35867dc\',action=\'15972885\' */\nselect \'Pair\' as entity, to_jsonb(c.*) as data from (select  * \n  from "sgd213744"."pair" c\n where c.block_range @> $1 and coalesce(upper("block_range"), 2147483647) > $2 and lower("block_range") <= $3 and (c."created_at_timestamp" >= $4::numeric and c."token_0" = $5)\n\n order by "created_at_timestamp" asc, "id" asc\n limit 1000) c -- binds: [15972885, 15972885, 15972885, "1604158136", "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"]'}
retry
created_at 1604158136
len 1000
created_at 1606678827
len 1000
created_at 1613250431
len 1000
created_at 1619758213
len 1000
created_at 1622475131
len 1000
created_at 1623884729
len 1000
creat

KeyboardInterrupt: 

In [5]:
def fetch_pair_hour_data(client, hour_start_unix=0, pair_ids=None):
    tmpl = """
    {{
      pairHourDatas(orderBy: hourStartUnix, first: 1000, where: {{ hourStartUnix_gte: {}, pair_in: [{}], }}) {{
        id
        hourStartUnix
        pair {{
          id
        }}
        reserve0
        reserve1
        reserveUSD
        hourlyVolumeToken0
        hourlyVolumeToken1
        hourlyVolumeUSD
        hourlyTxns
      }}
    }}
    """
    pair_ids_str = ','.join(['"{}"'.format(x) for x in pair_ids])
    return execute_query(client, tmpl.format(hour_start_unix, pair_ids_str))

def do_fetch_all_pair_hour_datas(client, pair_ids=None, raise_error=False, hour_start_unix=None, end_unix=None):
    dfs = []
    while hour_start_unix < end_unix:
        print('pair_ids hour_start_unix end_unix', pair_ids[0], pair_ids[-1], hour_start_unix, end_unix)
        try:
            res = fetch_pair_hour_data(client, hour_start_unix=hour_start_unix, pair_ids=pair_ids)
        except Exception as e:
            if raise_error:
                raise
            print(e)
            print('retry')
            time.sleep(10)
            continue
        print('len', len(res['pairHourDatas']))
        if len(res['pairHourDatas']) == 0:
            break
        for row in res['pairHourDatas']:
            row['pair_id'] = row['pair']['id']
        df = pd.DataFrame(res['pairHourDatas']).drop(columns=['pair'])
        dfs.append(df)
        hour_start_unix = df['hourStartUnix'].max()
        if len(res['pairHourDatas']) < 1000:
            break
            
    if len(dfs) == 0:
        return None

    df = pd.concat(dfs).drop_duplicates()
    df = df.rename(columns={
        'hourStartUnix': 'timestamp'
    }).drop(columns=['id'])
    df['timestamp'] = df['timestamp'].astype('int')
    df = df.loc[df['timestamp'] < end_unix]
    df = df.set_index(['timestamp', 'pair_id']).sort_index()
    return df

def fetch_all_pair_hour_datas(client, pair_ids=None, hour_start_unix=None, end_unix=None):
    dfs = []
    for i in range(0, len(pair_ids), 100):
        df = do_fetch_all_pair_hour_datas(
            client,
            pair_ids=pair_ids[i:min(i + 100, len(pair_ids))], 
            raise_error=False,
            hour_start_unix=hour_start_unix,
            end_unix=end_unix
        )
        if df is None:
            continue
        dfs.append(df)
    df = pd.concat(dfs)
    df = df.sort_index()
    return df

transport = RequestsHTTPTransport(url="https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2")
client = Client(transport=transport, fetch_schema_from_transport=True)

df_pair = pd.read_pickle('/tmp/df_pair.pkl')

pair_ids = sorted(df_pair.index.unique().tolist())
df_hour = fetch_all_pair_hour_datas(client, pair_ids, 1668000000, 1668000000 + 3600 * 24)

print('index duplicated count {}'.format(df_hour[df_hour.index.duplicated(keep='last')].shape[0]))
df_hour.to_pickle('/tmp/df_hour.pkl')
display(df_hour)

pair_ids hour_start_unix end_unix 0x002c755bc6b99f61d59a76047a13d6be41bb8fd1 0x18e229df96ca1e86bd0b90681effc3f76cbfedd3 1668000000 1668086400


KeyboardInterrupt: 

In [8]:
from google.cloud import bigquery

class HourFetcher:
    def __init__(self):
        self.keys = {}
        self.data_id = 'univ2_hour'
        
    def fetch(self, last_timestamp=None):
        transport = RequestsHTTPTransport(url="https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2")
        client = Client(transport=transport, fetch_schema_from_transport=True)
        
        # https://etherscan.io/block/10000834
        factory_deploy_at = 1588566790
        
        pair_ids = self._get_pair_ids()
        start_unix = 1 if last_timestamp is None else last_timestamp + 1
        end_unix = factory_deploy_at + 24 * 3600 if last_timestamp is None else start_unix + 24 * 3600
        df = fetch_all_pair_hour_datas(client, pair_ids, start_unix, end_unix)
        
        df = df.loc[df['timestamp'] < df['timestamp'].max()] # remove partial
        
        return df

    def _get_pair_ids(self):
        project_id = os.getenv('GC_PROJECT_ID')
        dataset_name = os.getenv('ALPHAPOOL_DATASET')
        table_id = f'{dataset_name}.univ2_pair'
        
        client = bigquery.Client(project=project_id)
        query = f'SELECT id FROM `{table_id}`'
        query_job = client.query(query)
        ids = []
        for row in query_job:
            ids.append(row['id'])
        
        return sorted(ids)

In [13]:
fetcher = HourFetcher()
# df = fetcher.fetch(last_timestamp=None)
# display(df)
df = fetcher.fetch(last_timestamp=(1668000000 // 3600) * 3600)
display(df)

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [9]:
fetchers = [
    PairFetcher(),
    HourFetcher(),
]

data = cloudpickle.dumps(fetchers)
data = lzma.compress(data)
with open('/home/jovyan/data/20221114_univ2.xz', 'wb') as f:
    f.write(data)